In [1]:
import sys
import os
script_dir = os.getcwd() # coleata o diretorio do projeto atual
sys.path.append(script_dir)

In [2]:
import pandas as pd
import numpy as np
import nltk
import random
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import OneHotEncoder
from tensorflow.keras import models, layers
from keras.models import clone_model
from gensim.models import Word2Vec
from nltk.tokenize import word_tokenize

In [3]:
data = pd.read_csv(r'CSV\tudo_junto_pp.csv')
data.head(5)

,Ano,Enunciado_Alternativas,Area_de_Conhecimento
0,2020,bienalsur una invitación cruz la front ded señ...,Linguagens
1,2020,españ corazón feder dim una gran sorpr hab pre...,Linguagens
2,2020,dispon httpinversorsaludcom acess 18 ago 2017 ...,Linguagens
3,2020,lo orígen habit expresíon che hay alg má argen...,Linguagens
4,2020,habl con acent andaluz telediari la 9 hac uno ...,Linguagens


In [4]:
areas = data['Area_de_Conhecimento'].unique()
areas

array(['Linguagens', 'Ciências Humanas', 'Ciências da Natureza',
       'Matemática'], dtype=object)

In [5]:
# Passo 1: Defina um mapeamento personalizado
mapa_personalizado = {
    'Linguagens': 0,
    'Ciências Humanas': 1,
    'Ciências da Natureza': 2,
    'Matemática': 3
}

# Passo 2: Substitua os valores na coluna usando o mapeamento
data['Area_de_Conhecimento'] = data['Area_de_Conhecimento'].map(mapa_personalizado)

In [6]:
areas = data['Area_de_Conhecimento'].unique()
areas

array([0, 1, 2, 3], dtype=int64)

In [7]:
# Divisão dos dados em treino e teste
X_train, X_test, y_train, y_test = train_test_split(data['Enunciado_Alternativas'], data['Area_de_Conhecimento'], test_size=0.2, random_state=1)
X_validation, y_validation = data['Enunciado_Alternativas'], data['Area_de_Conhecimento']

In [8]:
# Vetorização dos textos usando TF-IDF
vectorizer = TfidfVectorizer(max_features=5000)
X_train = vectorizer.fit_transform(X_train).toarray()
X_test = vectorizer.transform(X_test).toarray()
X_validation = vectorizer.transform(X_validation).toarray()


In [110]:
from copy import deepcopy

def create_MLP_model(input_dim, output_dim, dense_layers):
    model = models.Sequential()
    
    # Redimensionar os dados de entrada
    model.add(layers.Reshape((input_dim,), input_shape=(input_dim, 1)))
    
    model.add(layers.Dense(dense_layers[0], activation='relu'))

    for neurons in dense_layers[1:]:
        model.add(layers.Dense(neurons, activation='relu'))

    model.add(layers.Dense(output_dim, activation='softmax'))

    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

    return model

class Genetic_algoritm:
    def __init__(self, tamanho_populacao : int, numero_prog : int, alpha : float, MLPdim : tuple[3], data : tuple[4]) -> None:
        self.tamanho_populacao = tamanho_populacao
        self.numero_prog       = numero_prog
        self.alpha             = alpha
        self.MLPdim            = MLPdim
        self.populacao         = []

        self.X_train = data[0]
        self.Y_train = data[1]
        self.X_test  = data[2]
        self.Y_test  = data[3]

        self.iniciar_gen()
    
    def iniciar_gen(self) -> None:
        for x in range(self.tamanho_populacao): self.populacao.append([create_MLP_model(self.MLPdim[0], self.MLPdim[1], self.MLPdim[2]), 0])

    def pontuador(self, resposta_prevista : int, resposta_real : int) -> int:
        return 1 if resposta_prevista == resposta_real else 0
    
    def avaliador(self, individuo) -> int:
        pontuacao = 0
        resposta_prevista = individuo[0].predict(self.X_train)
        resposta_prevista = np.argmax(resposta_prevista, axis=1)   
        for posicao, ponto in enumerate(resposta_prevista):
            pontuacao += self.pontuador(ponto, self.Y_train[posicao])

        return pontuacao
    
    def avaliador_geracional(self) -> None:
        for index, individuo in enumerate(self.populacao): 
            self.populacao[index][1] = self.avaliador(self.populacao[index])
    
    def seleciona_progenitores(self) -> list:
        redes_branch_ordenada = sorted(self.populacao, key=lambda item: item[1], reverse=True)
        print(redes_branch_ordenada[:self.numero_prog])
        return redes_branch_ordenada[:self.numero_prog]
    
    def gerador_descendentes(self) -> None:
        lista_pais   = self.seleciona_progenitores()
        descendentes = self.tamanho_populacao - self.numero_prog
        prox_gen     = []
            
        for x in range(descendentes): 
            prox_gen.append(lista_pais[x % len(lista_pais)])
        for indiv in range(len(prox_gen)):
            prox_gen[indiv][0] = self.mutacao(prox_gen[indiv][0])
        prox_gen.extend(lista_pais)
        print(f"{len(prox_gen)}, {prox_gen}")
        self.populacao = prox_gen
    
    def mutacao(self, individuo):
        for layer in individuo.layers:
            if len(layer.get_weights()) > 0:
                pesos, biases = layer.get_weights()
                novos_pesos   = pesos + self.alpha * np.random.randn(*pesos.shape)
                
                layer.set_weights([novos_pesos, biases])

        return individuo
    
    def estrategia_evolutiva(self, n_geracoes : int) -> list:
        for x in range(n_geracoes):
            self.avaliador_geracional()
            self.gerador_descendentes()

            avaliador = sorted(self.populacao, key=lambda item: item[1], reverse=True)
            print(avaliador)
            print(len(avaliador))
            
        return self.populacao[:1]

In [111]:
y_train = [x for x in y_train]
y_test = [x for x in y_test]

In [112]:
# self, tamanho_populacao : int, numero_prog : int, alpha : float, MLPdim : tuple[3], data : tuple[4]

tamanho_populacao = 100
numero_prog = 6
alpha = 0.01
rede_neural = (X_train.shape[1], len(areas), [64, 32, 64])
data = (X_train, y_train, X_test, y_test)

algoritimo_genetico = Genetic_algoritm(tamanho_populacao, numero_prog, alpha, rede_neural, data)

c:\Users\Pichau\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\reshaping\reshape.py:39: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [113]:
algoritimo_genetico.estrategia_evolutiva(5)

19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step
19/19 ━━━━━━━━━━━━━━━━━━━━ 1s 33ms/step
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step
19/19 ━━━━━━━━━━━━━━━━━━

[[<Sequential name=sequential_1603, built=True>, 147]]

In [105]:
a = ["a","b"]
b = [1, 2]
a.extend(b)
print(a)

['a', 'b', 1, 2]
